In [1]:
import glob
import json
import os
import random as rnd
import shutil
from itertools import combinations

import albumentations as A
import keras.backend as K
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_ranking as tfr
import tensorflow_recommenders as tfrs
from IPython.core.interactiveshell import InteractiveShell
from sklearn.model_selection import StratifiedGroupKFold, train_test_split
from tensorflow import keras
from tensorflow.keras import Input, Model, activations, layers, losses
from tensorflow.keras.utils import Progbar, plot_model
from tensorflow.nn import leaky_relu
from tqdm import tqdm

from _data.artist_data.ny_baseline import eval_submission as ev_sub
from _data.artist_data.ny_baseline import get_ranked_list, inference
from src.test_utils import (
    TestGenerator,
    choose_100,
    eval_submission,
    load_submission,
    pairwise_distances,
)
from src.utils import make_callbacks, plot_history

InteractiveShell.ast_node_interactivity = "all"

In [2]:
class CFG:
    def __init__(
        self,
    ):
        self.seed = 39
        self.batch_size = 8
        self.img_size = (512, 60)
        self.n_folds = 8
        self.fold = 0
        self.norm = False
        self.n_blocks = 4
        self.emb_len = 1024
        self.kernel_size = (5, 2)
        self.batch_norm = False
        self.n_epochs = 300
        self.n_pairs_in_batch = 100
        self.input_shape = (self.img_size[0], self.img_size[1], 1)

In [3]:
cfg = CFG()

In [4]:
root_dir = "/app/_data/artist_data/"
mod_dir = f"/app/_data/artist_data/models/test_arch/tripl_semi_hard_8_{cfg.fold}/"
mod_dir

'/app/_data/artist_data/models/test_arch/tripl_semi_hard_8_0/'

In [5]:
train = pd.read_csv(os.path.join(root_dir, "train.csv"))
train = train[train["artistid_count"] != 1].reset_index(drop=True)
test = pd.read_csv(os.path.join(root_dir, "test_meta.tsv"), sep="\t")
test["path"] = test["archive_features_path"].apply(
    lambda x: os.path.join(root_dir, "test_features", x)
)

## train_val_split

In [6]:
gkf = StratifiedGroupKFold(n_splits=cfg.n_folds, shuffle=True, random_state=cfg.seed)
for n, (train_ids, val_ids) in enumerate(
    gkf.split(
        X=train[["artistid", "artistid_count"]],
        y=train["artistid_count"],
        groups=train["artistid"],
    )
):
    train.loc[val_ids, "fold"] = n

In [7]:
train_df = train[train["fold"] != cfg.fold].reset_index(drop=True)
val_df = train[train["fold"] == cfg.fold].reset_index(drop=True)

## DataGenerator

In [8]:
class DataGenerator:
    def __init__(
        self,
        data,
        img_size,
        batch_size=32,
        n_pairs_in_batch=50,
        norm=False,
        shuffle=True,
        transpose=True,
        augment=True,
    ):
        self.data = data.reset_index(drop=True)
        self.img_size = img_size
        self.batch_size = batch_size
        self.n_pairs_in_batch = n_pairs_in_batch
        self.norm = norm
        self.shuffle = shuffle
        self.transpose = transpose
        self.augment = augment
        self.artist_ids = self.data["artistid"].unique().tolist()
        self.artis2path = self.data.groupby("artistid").agg(list)["path"].to_dict()
        self.paths = self.data["path"].tolist()

        if self.shuffle:
            np.random.shuffle(self.artist_ids)

    def __len__(self):
        return self.data.shape[0]

    def augment_fn(self, img):
        transform = A.Compose(
            [
                A.RandomCrop(always_apply=True, p=1.0, height=512, width=60),
                A.Flip(p=0.2),
                A.PixelDropout(p=0.1, dropout_prob=0.01),
                A.CoarseDropout(
                    p=0.1,
                    max_holes=11,
                    max_height=5,
                    max_width=3,
                    min_holes=1,
                    min_height=2,
                    min_width=2,
                ),
                A.RandomGridShuffle(p=0.3, grid=(1, 6)),
            ]
        )
        return transform(image=img)["image"]

    def load_img(self, path):
        img = np.load(path).astype("float32")
        if self.norm:
            img -= img.min()
            img /= img.max()
        if self.augment:
            img = self.augment_fn(img)
        else:
            wpad = (img.shape[1] - self.img_size[1]) // 2
            img = img[:, wpad : wpad + self.img_size[1]]
        if self.transpose:
            img = img.transpose(1, 0)
        img = np.expand_dims(img, -1)
        return img

    def get_list(self, artis_id):
        valid_paths = self.artis2path[artis_id]
        np.random.shuffle(valid_paths)
        anchor_img = np.expand_dims(self.load_img(valid_paths[0]), 0)
        positive_imgs = np.array([self.load_img(p) for p in valid_paths[1:]])
        negative_paths = rnd.sample(
            [x for x in self.paths if x not in valid_paths],
            self.n_pairs_in_batch - len(positive_imgs),
        )
        negative_imgs = np.array([self.load_img(p) for p in negative_paths])
        imgs = np.concatenate([positive_imgs, negative_imgs])
        labels = np.full(self.n_pairs_in_batch, 10)
        labels[positive_imgs.shape[0] :] = 0
        perm = np.random.permutation(self.n_pairs_in_batch)
        imgs = imgs[perm]
        labels = labels[perm]
        imgs = np.concatenate([anchor_img, imgs])
        return imgs, labels

    def __call__(self):
        np.random.shuffle(self.artist_ids)
        ix = 0
        while ix < len(self.artist_ids):
            tracks, labels = self.get_list(self.artist_ids[ix])
            tracks = tf.convert_to_tensor(tracks)
            labels = tf.convert_to_tensor(labels)
            yield tracks, labels
            ix += 1

In [9]:
train_gen = DataGenerator(
    data=train_df,
    img_size=cfg.img_size,
    batch_size=cfg.batch_size,
    n_pairs_in_batch=cfg.n_pairs_in_batch,
    norm=False,
    shuffle=True,
    transpose=False,
    augment=True,
)
val_gen = DataGenerator(
    data=val_df,
    img_size=cfg.img_size,
    batch_size=cfg.batch_size,
    n_pairs_in_batch=cfg.n_pairs_in_batch,
    norm=False,
    shuffle=True,
    transpose=False,
    augment=False,
)

In [10]:
for n, (a, b) in enumerate(train_gen.__call__()):
    print(a.shape, b.shape)
    if n == 1:
        break

(101, 512, 60, 1) (100,)
(101, 512, 60, 1) (100,)


In [11]:
train_ds = tf.data.Dataset.from_generator(
    train_gen,
    output_signature=(
        tf.TensorSpec(
            shape=(cfg.n_pairs_in_batch + 1, *cfg.img_size, 1),
            dtype=tf.float32,
            name="tracks",
        ),
        tf.TensorSpec(
            shape=(cfg.n_pairs_in_batch),
            dtype=tf.float32,
            name="labels",
        ),
    ),
)
val_ds = tf.data.Dataset.from_generator(
    val_gen,
    output_signature=(
        tf.TensorSpec(
            shape=(cfg.n_pairs_in_batch + 1, *cfg.img_size, 1),
            dtype=tf.float32,
            name="tracks",
        ),
        tf.TensorSpec(
            shape=(cfg.n_pairs_in_batch),
            dtype=tf.float32,
            name="labels",
        ),
    ),
)
train_ds

<FlatMapDataset element_spec=(TensorSpec(shape=(101, 512, 60, 1), dtype=tf.float32, name='tracks'), TensorSpec(shape=(100,), dtype=tf.float32, name='labels'))>

In [12]:
a = train_ds.take(1)

In [13]:
a

<TakeDataset element_spec=(TensorSpec(shape=(101, 512, 60, 1), dtype=tf.float32, name='tracks'), TensorSpec(shape=(100,), dtype=tf.float32, name='labels'))>

In [14]:
for example in train_ds.take(1):
    print(example[0].shape, example[1].shape)

(101, 512, 60, 1) (100,)


In [15]:
def embNet(
    input_shape,
    kernel_size=3,
    dropout_rate=0.1,
    embedding_len=1024,
    activation_fn="relu",
    padding="same",
    dense_activation="relu",
):
    base_model = tf.keras.models.Sequential(
        [
            keras.layers.Conv1D(
                filters=1,
                kernel_size=kernel_size,
                activation=activation_fn,
                input_shape=input_shape,
                padding=padding,
                name="conv_1",
            ),
            keras.layers.Conv1D(
                filters=1,
                kernel_size=kernel_size,
                activation=activation_fn,
                padding=padding,
                name="conv_2",
            ),
            keras.layers.Dropout(rate=dropout_rate, name="dropout1"),
            keras.layers.MaxPooling2D(
                pool_size=(2, 2), strides=1, padding="same", name="max_1"
            ),
            keras.layers.Conv1D(
                filters=1,
                kernel_size=kernel_size,
                activation=activation_fn,
                padding=padding,
                name="conv_3",
            ),
            keras.layers.Conv1D(
                filters=1,
                kernel_size=kernel_size,
                activation=activation_fn,
                padding=padding,
                name="conv_4",
            ),
            keras.layers.Flatten(name="flatten_base"),
            keras.layers.Dense(embedding_len * 2, activation="relu", name="dense_1"),
            keras.layers.Dense(embedding_len, activation=None, name="dense_base_2"),
        ]
    )
    embedding_net = keras.Model(
        inputs=base_model.input, outputs=base_model.output, name="embedding"
    )
    return embedding_net

In [16]:
embedding_net = embNet(
    input_shape=(512, 60, 1),
    kernel_size=3,
    dropout_rate=0.1,
    embedding_len=1024,
    activation_fn="relu",
    padding="valid",
    dense_activation="relu",
)

In [17]:
emb = embedding_net(example[0])
emb.shape

TensorShape([101, 1024])

In [80]:
anchors_embeddings = emb[:1]
img_embeddings = emb[1:]
anchors_embeddings.shape
list_length = img_embeddings.shape[0]
anchors_embedding_repeated = tf.repeat(anchors_embeddings, [list_length], axis=0)

TensorShape([1, 1024])

In [81]:
anchors_embedding_repeated.shape

TensorShape([30, 1024])

In [82]:
anchors_embedding_repeated.shape

concatenated_embeddings = tf.concat([anchors_embedding_repeated, img_embeddings], 1)

concatenated_embeddings.shape

TensorShape([30, 1024])

TensorShape([30, 2048])

In [83]:
score_model = tf.keras.Sequential(
    [
        # Learn multiple dense layers.
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(64, activation="relu"),
        # Make rating predictions in the final layer.
        tf.keras.layers.Dense(1),
    ]
)

scores = score_model(concatenated_embeddings)

scores.shape

scores[0, :]

TensorShape([30, 1])

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>

In [84]:
loss = tfr.keras.losses.ListMLELoss()

In [86]:
task = tfrs.tasks.Ranking(
    loss=loss,
    metrics=[
        tfr.keras.metrics.NDCGMetric(name="ndcg_metric"),
        tf.keras.metrics.RootMeanSquaredError(),
    ],
)

In [90]:
tf.squeeze(scores, axis=-1)

<tf.Tensor: shape=(30,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

In [92]:
example[1]

<tf.Tensor: shape=(30,), dtype=float32, numpy=
array([ 0., 10.,  0.,  0.,  0.,  0., 10.,  0.,  0.,  0.,  0.,  0.,  0.,
        0., 10.,  0., 10.,  0.,  0.,  0., 10.,  0., 10., 10.,  0.,  0.,
        0., 10., 10.,  0.], dtype=float32)>

In [95]:
tf.reduce_min(input_tensor=example[1], axis=0, keepdims=True)

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>

In [96]:
task(
    labels=tf.expand_dims(example[1], 0),
    predictions=tf.expand_dims(tf.squeeze(scores, axis=-1),0),
)

<tf.Tensor: shape=(), dtype=float32, numpy=74.65823>

In [18]:
class RankingModel(tfrs.Model):
    def __init__(self, loss, emb_model):
        super().__init__()
        embedding_dimension = 32

        # embeddings model
        self.embeddings = emb_model

        # Compute predictions.
        self.score_model = tf.keras.Sequential(
            [
                # Learn multiple dense layers.
                tf.keras.layers.Dense(256, activation="relu"),
                tf.keras.layers.Dense(64, activation="relu"),
                # Make rating predictions in the final layer.
                tf.keras.layers.Dense(1),
            ]
        )

        self.task = tfrs.tasks.Ranking(
            loss=loss,
            metrics=[
                tfr.keras.metrics.NDCGMetric(name="ndcg_metric"),
                tf.keras.metrics.RootMeanSquaredError(),
            ],
        )

    def call(self, features):
        # batch_anchors = features[:1]
        # batch_tracks = features[1:]
        # We first convert the id features into embeddings.
        # User embeddings are a [batch_size, embedding_dim] tensor.
        embeddings = self.embeddings(features)
        anchors_embeddings = embeddings[:1, ...]
        img_embeddings = embeddings[1:, ...]
        # anchors_embeddings = self.embeddings(batch_anchors)

        # Movie embeddings are a [batch_size, num_movies_in_list, embedding_dim]
        # tensor.
        # img_embeddings = tf.convert_to_tensor(
        #     [self.embeddings(x) for x in batch_tracks]
        # )

        # We want to concatenate user embeddings with movie emebeddings to pass
        # them into the ranking model. To do so, we need to reshape the user
        # embeddings to match the shape of movie embeddings.

        list_length = img_embeddings.shape[0]
        anchors_embedding_repeated = tf.repeat(
            anchors_embeddings, [list_length], axis=0
        )

        # Once reshaped, we concatenate and pass into the dense layers to generate
        # predictions.
        concatenated_embeddings = tf.concat(
            [anchors_embedding_repeated, img_embeddings], 1
        )

        return self.score_model(concatenated_embeddings)

    def compute_loss(self, features, training=False):
        features, labels = features
        # labels = features["labels"]
        scores = self(features)

        return self.task(
            labels=tf.expand_dims(labels, 0),
            predictions=tf.expand_dims(tf.squeeze(scores, axis=-1), 0),
        )

In [19]:
listwise_model = RankingModel(tfr.keras.losses.ListMLELoss(), embedding_net)
listwise_model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [21]:
mse_model = RankingModel(tf.keras.losses.MeanSquaredError(), embedding_net)
mse_model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [ ]:
history = mse_model.fit(
    x=train_ds,
    epochs=10,
    verbose="auto",
    callbacks=None,
    validation_data=val_ds,
    shuffle=True,
    steps_per_epoch=train_gen.__len__(),
    validation_steps=val_gen.__len__(),
    validation_freq=1,
    max_queue_size=100,
    workers=64,
    use_multiprocessing=False,
)

Epoch 1/10
  2125/146356 [..............................] - ETA: 5:52:24 - ndcg_metric: 0.4224 - root_mean_squared_error: 2.7286 - loss: 7.4450 - regularization_loss: 0.0000e+00 - total_loss: 7.4450